## Action1：不用任何数学库，如何求出sqrt(10)，并且精确到小数点后10位

In [21]:
import math

In [22]:
def sqrt(n,EPSION): #求出sqrt(n),n属于[1,10]
    low, high=1, 100
    mid = (low+high)/2

    while high-low > EPSION:
        if mid**2 > n:
            high = mid
        else:
            low = mid
        mid=(high+low)/2
    return mid

In [23]:
result=sqrt(10,1e-10)

In [24]:
print('根号10的结果为:', result)

根号10的结果为: 3.162277660176642


In [25]:
result2=math.sqrt(10)
print('math包的根号10结果为:',result2)

math包的根号10结果为: 3.1622776601683795


## Action2：汽车投诉信息采集

In [8]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
import json

In [9]:
#随机生成User-Agent
user = UserAgent()
user_random = user.random

In [10]:
# 根据request_url得到soup
def get_page_content(request_url):
    # 得到页面的内容
    headers = {"User-Agent": user_random}
    html=requests.get(request_url,headers=headers,timeout=10)
    content = html.text
    #print(content)

    # 通过content创建BeautifulSoup对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

In [11]:
def get_dict_content(dict_url): #得到问题代码与中文名的对应字典
    headers = {"User-Agent": user_random}
    html=requests.get(dict_url,headers=headers,timeout=10)
    content = html.text
    #print(content)
    response = content[content.find('['):len(content)]
    index2word = json.loads(response)
    index2word_dict={}
    for i in index2word:
        for j in range(len(i['items'])):
            index2word_dict[i['value']+str(i['items'][j]['id'])]=i['name']+i['items'][j]['title']
    return index2word_dict

In [12]:
# 提取BeautifulSoup对象的内容生成DataFrame
def analysis(soup):
    # 提取需要的内容(这里提取标签<div class="tslb_b">)
    temp = soup.find('div',class_="tslb_b")
    # 创建DataFrame
    df = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
    tr_list = temp.find_all('tr') #找到提取内容中的所有<tr>标签生成列表(每行都是一个<tr>标签)
    for tr in tr_list[1:]:
        temp={}
        td_list=tr.find_all('td') #提取每个<tr>标签中的所有<td>标签
        
        problem=[index2word_dict[i] for i in td_list[5].text.split(',')[:-1]]

        if len(td_list) > 0:
            id, brand, car_model, type, desc, problem, datetime, status=td_list[0].text, td_list[1].text, td_list[2].text, td_list[3].text, td_list[4].text, problem, td_list[6].text, td_list[7].text
            #print(id, brand, car_modle, type, desc, problem, datetime, status)
            temp['id'], temp['brand'], temp['car_model'], temp['type'], temp['desc'], temp['problem'], temp['datetime'], temp['status']= id, brand, car_model, type, desc, problem, datetime, status
            df = df.append(temp, ignore_index=True)
    return df

In [13]:
#对多个页面进行解析
page_num = 20
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'
dict_url='http://www.12365auto.com/js/cTypeInfo.js?version=20200823'

index2word_dict=get_dict_content(dict_url) 

result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
for i in range(page_num):
    request_url = base_url+str(i+1)+'.shtml'
    soup = get_page_content(request_url)
    #print(request_url)
    df = analysis(soup)
    #print(df)
    result = result.append(df)

/Users/muyu/opt/anaconda3/envs/NLP/lib/python3.7/site-packages/bs4/__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [14]:
result.head(10)

,id,brand,car_model,type,desc,problem,datetime,status
0,515601,东风启辰,启辰T60,2018款 1.6L CVT 智悦版(ISS) 国VI,启辰T60多次电瓶亏电无法启动希望处理,[车身附件及电器电瓶故障],2020-08-29,信息审核
1,515594,长安马自达,马自达3 昂克赛拉,2020款 1.5L 自动 质美版,长安马自达昂克赛拉发动机变速箱漏油\t,"[发动机漏油, 变速器漏油, 制动系统异响]",2020-08-29,信息审核
2,515592,一汽-大众,探岳,2020款 330TSI 两驱 豪华智联版,一汽大众探岳油泵异响希望尽快得到处理,[车身附件及电器汽油泵故障],2020-08-29,信息审核
3,515589,一汽-大众奥迪,奥迪A4L,2018款 2.0T 时尚型 30周年年型,一汽大众奥迪A4L中控屏背面漆面起泡,[车身附件及电器漆面起泡开裂],2020-08-29,信息审核
4,515577,吉利汽车,缤瑞,2018款 14T CVT 缤纷版,吉利缤瑞仪表台异响 承诺洗车服务未兑现,"[车身附件及电器中控台异响, 承诺不兑现服务承诺不兑现]",2020-08-29,信息审核
5,515559,一汽-大众,高尔夫,2014款 1.6L 手动 时尚型,一汽大众高尔夫车内异味越来越严重,[车身附件及电器车内异味],2020-08-29,信息审核
6,515543,北京现代,名图,2017款 1.6T 自动 智能型 GLS,北京现代名图行驶中方向跑偏四轮定位未解决,[前后桥及悬挂系统跑偏],2020-08-29,信息审核
7,515538,众泰汽车,云100,2014款 云100 舒适型,众泰云100动力电池故障要求尽快解决,[车身附件及电器动力电池故障],2020-08-29,信息审核
8,515537,东风日产,逍客,2016款 2.0L CVT 精英版,东风日产逍客变速箱阀体损坏不予更换,"[变速器故障灯亮, 变速器滑阀箱故障]",2020-08-29,信息审核
9,515535,上汽大众,朗逸,2013款 1.6L 自动 舒适版 改款,上汽大众朗逸发动机水泵处漏油要求解决,[发动机漏油],2020-08-29,信息审核
